In [71]:
import findspark
findspark.init('/usr/local/spark/spark-2.3.2-bin-hadoop2.7')

import pyspark
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as f
from pyspark.sql.types import DateType, StringType
from pyspark.sql import *
spark = SparkSession \
    .builder \
    .appName("Poolminers") \
    .getOrCreate()

from datetime import datetime


sc=SparkContext.getOrCreate(spark)


In [5]:
import timeit

In [90]:
# read transactions
df_tx = spark.read.parquet('../data/transaction.parquet')

In [87]:
# read blocks
def explode_block(df,col):
    # explode the list the first time
    df = df.withColumn(col,explode(split(f.col(col),'\],\[') ))
    # explode the
    #tx_hash = udf(lambda x: x.replace([],''))
    #df = df.withColumn('col',tx_hash(df[col]))
    # extract the transaction_hash
    df = df.withColumn(col,regexp_replace('transaction_list', '(\[|\]|")', ''))
    df = df.withColumn(col, df[col].substr(0, 64))
    return df
    
df_block = spark.read.parquet('../data/block.parquet')
df_block_1 = explode_block(df_block,'transaction_list')
#display entire cell
#df_block_1.select(f.col('transaction_list')).show(50,False)

In [126]:
# slice dataframe
# dateformat = 'yyyy-mm-dd 00:00:00'
def date_to_timestamp(date):
    return datetime.strptime(date, "%Y-%m-%d %H:%M:%S").timestamp()

startdate = '2018-09-01 00:00:00'
enddate = '2018-09-07 00:00:00'
def truncate_dataframe(df,startdate,enddate):
    startdate = date_to_timestamp(startdate)
    enddate = date_to_timestamp(enddate)
    if startdate > enddate:
        startdate = enddate
    df = df.filter((f.col('block_timestamp') >= startdate) & 
              (f.col('block_timestamp') <= enddate))
    df = timestamp_to_date(df,'block_timestamp')
    return df

def timestamp_to_date(df,col):
    return df.withColumn('block_timestamp', f.from_unixtime('block_timestamp').cast(DateType()))
 
# truncate dataframes
df_block_2 = truncate_dataframe(df_block_1,startdate, enddate).drop('__index_level_0__')
df_tx_1 = truncate_dataframe(df_tx,startdate,enddate).drop('block_timestamp')
#



In [128]:
# join dataframes
df_pool_warehouse = df_block_2.join(df_tx_1, df_block_2.transaction_list == 
                                    df_tx_1.transaction_hash,how='left').drop('transaction_list')


In [129]:
# WRITE TO PARQUET
df_pool_warehouse = df_pool_warehouse.drop('__index_level_0__')
df_pool_warehouse.write.parquet("../data/pool_warehouse.parquet",compression='gzip')

In [130]:
print(df_pool_warehouse.count())

92521
